This notebook provides examples to go along with the [textbook](https://underactuated.csail.mit.edu/dp.html).  I recommend having both windows open, side-by-side!


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm
from pydrake.all import (DiagramBuilder, LogVectorOutput, MathematicalProgram,
                         Polynomial, Simulator, Solve,
                         SymbolicVectorSystem, Variable, Variables)
from pydrake.examples import PendulumParams

plt.rcParams.update({"savefig.transparent": True})

# Sums-of-squares Dynamic Programming

## Cubic polynomial optimal control

In [ ]:
# Scalar dynamics.
f = lambda x, u : x - 4 * x ** 3 + u
# Quadratic running cost.
l = lambda x, u : x ** 2 + u ** 2
# Input limits.
U = [-1, 1]
# State limits (region of state space where we approximate the value function).
X = [-1, 1]

# Numerically integrate the solution to the HJB to get the "true" optimal
# cost-to-go; this only works in the scalar case when we can compute the
# optimal policy explicitly as a function of dJdx. TODO: Use Drake's
# InitialValueProblem after cleanup proposed in
# https://github.com/RobotLocomotion/drake/issues/12857 happens.
def optimal_cost_to_go():
    x = Variable('x')
    J = Variable('J')
    builder = DiagramBuilder()
    sys = builder.AddSystem(
        SymbolicVectorSystem(time=x,
                             state=[J],
                             dynamics=[
                                 2 * (x - 4 * x**3)
                                 + 2 * x * np.sqrt(2 - 8 * x**2 + 16 * x**4)
                             ], output=[J]))
    logger = LogVectorOutput(sys.get_output_port(), builder)
    diagram = builder.Build()
    simulator = Simulator(diagram)
    context = simulator.get_mutable_context()
    # Set J(0) = 0
    context.SetTime(0.0)
    context.SetContinuousState([0.0])
    simulator.AdvanceTo(1.0)
    log = logger.FindLog(context)
    return log.sample_times(), log.data()


# Plot dynamics with zero input.
n_breaks = 101
x_breaks = np.linspace(*X, n_breaks)
plt.plot(x_breaks, f(x_breaks, np.zeros(n_breaks)))
plt.xlabel(r'$x$')
plt.ylabel(r'$f(x, u=0)$')
plt.grid(True)

In [ ]:


# Given the degree for the approximate value function and the polynomials
# in the S procedure, solves the SOS and returns the approximate value function
# (together with the objective of the SOS program).
def sos_dp(deg):
    
    # Set up SOS program.
    prog = MathematicalProgram()
    x = prog.NewIndeterminates(1, 'x')[0]
    u = prog.NewIndeterminates(1, 'u')[0]
    J = prog.NewFreePolynomial(Variables([x]), deg)

    # Maximize volume beneath the value function.
    J_int = J.Integrate(x, -1, 1).ToExpression()
    prog.AddLinearCost(- J_int)
    
    # S-procedure for the input limits.
    xu = Variables([x, u])
    lamx = prog.NewSosPolynomial(xu, deg)[0]
    S_procedure = lamx * Polynomial((x - X[0]) * (X[1] - x))
    
    # S-procedure for the input limits.
    lamu = prog.NewSosPolynomial(xu, deg)[0]
    S_procedure += lamu * Polynomial((u - U[0]) * (U[1] - u))
    
    # Enforce Bellman inequality.
    J_dot = J.Differentiate(x) * Polynomial(f(x, u))
    prog.AddSosConstraint(J_dot + Polynomial(l(x, u)) - S_procedure)

    # J(0) = 0.
    prog.AddLinearConstraint(J.EvaluatePartial({x: 0}).ToExpression() == 0)

    # Solve and retrieve result.
    result = Solve(prog)
    assert result.is_success()

    # retrieve value function
    J_opt_expr = result.GetSolution(J.ToExpression())
    J_opt = lambda x_eval: J_opt_expr.Evaluate({x: x_eval})
    cost = - result.get_optimal_cost()
    
    return J_opt, cost

# Solve for increasing degree.
degrees = np.arange(1, 5) * 2
J = {deg: sos_dp(deg) for deg in degrees}

# Plot solution.
x_opt, J_opt = optimal_cost_to_go()
plt.figure()
plt.plot(x_opt, J_opt.T, 'k', label='J*')
plt.plot(-x_opt, J_opt.T, 'k')
for deg in degrees:
    label = f'Deg. {deg}'
    J_plot = [J[deg][0](xi) for xi in x_breaks]
    plt.plot(x_breaks, J_plot, label=label)
    plt.xlabel(r'$x$')
    plt.ylabel(r'$v$')
    plt.title('Value-function lower bound')
    plt.legend()
    plt.grid(True)

In [ ]:
# System dimensions. Here:
# x = [theta, theta_dot]
# z = [sin(theta), cos(theta), theta_dot]
nx = 2
nz = 3
nu = 1

# Map from original state to augmented state.
# Uses sympy to be able to do symbolic integration later on.
x2z = lambda x : np.array([np.sin(x[0]), np.cos(x[0]), x[1]])

# System dynamics in augmented state (z).
params = PendulumParams()
inertia = params.mass() * params.length() ** 2
tau_g = params.mass() * params.gravity() * params.length()
def f(z, u):
    return [
        z[1] * z[2],
        - z[0] * z[2],
        (tau_g * z[0] + u[0] - params.damping() * z[2]) / inertia
    ]

# State limits (region of state space where we approximate the value function).
x_max = np.array([np.pi, 2*np.pi])
x_min = - x_max
z_max = x2z(x_max)
z_min = x2z(x_min)

# Equilibrium point in both the system coordinates.
x0 = np.array([0, 0])
z0 = x2z(x0)
    
# Quadratic running cost in augmented state.
Q = np.diag([1, 1, 1])
R = np.diag([5])
def l(z, u):
    return (z - z0).dot(Q).dot(z - z0) + u.dot(R).dot(u)

Note: Use deg=10 for a nice swing-up, but the open-source solver (C-SDP) only seems to be able to handle degree up to 5.

In [ ]:
# Given the degree for the approximate value function and the polynomials
# in the S procedure, solves the SOS and returns the approximate value function
# (together with the objective of the SOS program).
def sos_dp(deg):

    # Set up optimization.
    prog = MathematicalProgram()
    z = prog.NewIndeterminates(nz, 'z')
    u = prog.NewIndeterminates(nu, 'u')
    J = prog.NewFreePolynomial(Variables(z), deg)
    J_expr = J.ToExpression()

    # Maximize volume beneath the value function.
    obj = J
    for i in range(nz):
        obj = obj.Integrate(z[i], z_min[i], z_max[i])
    prog.AddLinearCost(- obj.ToExpression())

    # S procedure for s^2 + c^2 = 1.
    lam = prog.NewFreePolynomial(Variables(z), deg).ToExpression()
    S_procedure = lam * (z[0]**2 + z[1]**2 - 1)

    # Enforce Bellman inequality.
    J_dot = J_expr.Jacobian(z).dot(f(z, u))
    prog.AddSosConstraint(J_dot + l(z, u) + S_procedure)

    # J(z0) = 0.
    J0 = J_expr.EvaluatePartial(dict(zip(z, z0)))
    prog.AddLinearConstraint(J0 == 0)

    # Solve and retrieve result.
    result = Solve(prog)
    assert result.is_success()
    J_star = Polynomial(result.GetSolution(J_expr))

    # Solve for the optimal feedback in augmented coordinates.
    Rinv = np.linalg.inv(R)
    f2 = np.array([[0], [0], [1 / inertia]])
    dJdz = J_star.ToExpression().Jacobian(z)
    u_star = - .5 * Rinv.dot(f2.T).dot(dJdz.T)

    return J_star, u_star, z

J_star, u_star, z = sos_dp(deg=4)

X1, X2 = np.meshgrid(np.linspace(x_min[0], x_max[0], 51),
                     np.linspace(x_min[1], x_max[1], 51))
X = np.vstack((X1.flatten(), X2.flatten()))
Z = x2z(X)
J = np.zeros(Z.shape[1])
for i in range(Z.shape[1]):
    J[i] = J_star.Evaluate({z[0]: Z[0, i], z[1]: Z[1, i], z[2]: Z[2, i]})

fig = plt.figure(figsize=(9, 4))
ax = fig.subplots()
ax.set_xlabel("q")
ax.set_ylabel("qdot")
ax.set_title("Cost-to-Go")
ax.imshow(J.reshape(X1.shape),
        cmap=cm.jet, aspect='auto',
        extent=(x_min[0], x_max[0], x_min[1], x_max[1]))
ax.invert_yaxis()
